In [1]:
import tensorflow as tf
import tensorflow_ranking as tfr

tf.enable_eager_execution()
tf.executing_eagerly()

True

In [2]:
! wget -O "/tmp/train.txt" "https://raw.githubusercontent.com/tensorflow/ranking/master/tensorflow_ranking/examples/data/train.txt"
! wget -O "/tmp/test.txt" "https://raw.githubusercontent.com/tensorflow/ranking/master/tensorflow_ranking/examples/data/test.txt"

--2019-04-26 17:52:10--  https://raw.githubusercontent.com/tensorflow/ranking/master/tensorflow_ranking/examples/data/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.72.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.72.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29788 (29K) [text/plain]
Saving to: ‘/tmp/train.txt’

/tmp/train.txt      100%[===================>]  29.09K  --.-KB/s    in 0.04s   

2019-04-26 17:52:11 (745 KB/s) - ‘/tmp/train.txt’ saved [29788/29788]

--2019-04-26 17:52:11--  https://raw.githubusercontent.com/tensorflow/ranking/master/tensorflow_ranking/examples/data/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.72.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.72.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8959 (8.7K) [text/plain]
Saving to: ‘/tmp/test.txt’

/tmp/tes

In [3]:
# Store the paths to files containing training and test instances.
# As noted above, we will assume the data is in the LibSVM format
# and that the content of each file is sorted by query ID.
_TRAIN_DATA_PATH="/tmp/train.txt"
_TEST_DATA_PATH="/tmp/test.txt"

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
_LOSS="pairwise_logistic_loss"

# In the TF-Ranking framework, a training instance is represented
# by a Tensor that contains features from a list of documents
# associated with a single query. For simplicity, we fix the shape
# of these Tensors to a maximum list size and call it "list_size,"
# the maximum number of documents per query in the dataset.
# In this demo, we take the following approach:
#   * If a query has fewer documents, its Tensor will be padded
#     appropriately.
#   * If a query has more documents, we shuffle its list of
#     documents and trim the list down to the prescribed list_size.
_LIST_SIZE=100

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.
_NUM_FEATURES=136

# Parameters to the scoring function.
_BATCH_SIZE=32
_HIDDEN_LAYER_DIMS=["20", "10"]

In [29]:
path =_TRAIN_DATA_PATH

train_dataset = tf.data.Dataset.from_generator(
        tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
      
    output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
    
    output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
  )

In [30]:
train_dataset = train_dataset.shuffle(1000).repeat().batch(_BATCH_SIZE)
train_dataset = train_dataset.make_one_shot_iterator().get_next()

In [31]:
feature_names = [
      "%d" % (i + 1) for i in range(0, _NUM_FEATURES)
]

In [33]:
{
    name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
}    

{'1': NumericColumn(key='1', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 '2': NumericColumn(key='2', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 '3': NumericColumn(key='3', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 '4': NumericColumn(key='4', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 '5': NumericColumn(key='5', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 '6': NumericColumn(key='6', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 '7': NumericColumn(key='7', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 '8': NumericColumn(key='8', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 '9': NumericColumn(key='9', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=None),
 '10': NumericColumn(key='10', shape=(1,), default_value=(0.0,), dtype=tf.float32,

In [5]:
def example_feature_columns():
    feature_names = [
      "%d" % (i + 1) for i in range(0, _NUM_FEATURES)
  ]
    return {
      name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
  }

def make_score_fn():

    def _score_fn(context_features, group_features, mode, params, config):
        del params
        del config
    # Define input layer.
    example_input = [
        tf.layers.flatten(group_features[name])
        for name in sorted(example_feature_columns())
    ]
    input_layer = tf.concat(example_input, 1)

    cur_layer = input_layer
    for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
        cur_layer = tf.layers.dense(
          cur_layer,
          units=layer_width,
          activation="tanh")

    logits = tf.layers.dense(cur_layer, units=1)
    
    return logits

    return _score_fn

In [6]:
def eval_metric_fns():
    metric_fns = {}
    metric_fns.update({
      "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [1, 3, 5, 10]
  })

    return metric_fns

In [13]:
def get_estimator(hparams):

    def _train_op_fn(loss):
    
        return tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.train.get_global_step(),
        learning_rate=hparams.learning_rate,
        optimizer="Adagrad")

    ranking_head = tfr.head.create_ranking_head(
      loss_fn=tfr.losses.make_loss_fn(_LOSS),
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

    return tf.estimator.Estimator(
      model_fn=tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          group_size=1,
          transform_fn=None,
          ranking_head=ranking_head),
      params=hparams)


In [14]:
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams)

NameError: name 'group_features' is not defined

In [15]:
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100)


NameError: name 'ranker' is not defined

In [16]:
ranker.evaluate(input_fn=lambda: input_fn(_TEST_DATA_PATH), steps=100)


NameError: name 'ranker' is not defined

In [17]:
ranker.model_dir


NameError: name 'ranker' is not defined